### Que 5 - Report

The k-center cost obtained by using greedy k-center algorithm on all the data points ($n = 8950$)  -

| k  	| Cost               	|
|----	|--------------------	|
| 2  	| 2.0106026585562478 	|
| 4  	| 1.789436782259935  	|
| 10 	| 1.4984577152624328 	|


To find optimal cost, the algorithm I have used is brute force i.e. I try all possible centers, calculate the cost and then take minimum cost among all. The time complexity of this algorithm is $O(n^{k+1}kd)$, where $d$ is the dimension of data set. Hence, with $n = 8950$ and $k = 2$ or $4$, this algorithm may take up hours, days or even centuaries to finish. 

Hence, to apply this algorithm, I choose random $20$ (By putting $n=20$ and $k=4$ the algorithm gives results in few seconds) points from the $8950$ points.

The k-center cost obtained by optimal algorithm with these 20 data points -

| k  	| Cost               	|
|----	|--------------------	|
| 2  	| 1.258411509001605 	|
| 4  	| 1.00772537014411  	|


The k-center cost obtained by using greedy k-center algorithm with these 20 data points -

| k  	| Cost               	|
|----	|--------------------	|
| 2  	| 1.679758268511973 	|
| 4  	| 1.2612759905672448  	|


Hence, approximation factor calculated as cost by greedy k-center on 20 data points / optimal cost on 20 data points -

| k  	| Factor               	|
|----	|--------------------	|
| 2  	| 1.3348243054806888 	|
| 4  	| 1.251606864265882  	|

As, expected this value lies in range [1, 2] since greedy k-center algorithm is 2-approximation of the k-center problem.

Now, approximation factor calculated as cost by greedy k-center on 8950 data points / optimal cost on 20 data points -

| k  	| Factor               	|
|----	|--------------------	|
| 2  	| 1.597730666140692 	|
| 4  	| 1.775718698045715  	|


In this case, since for calculation of optimal cost (denominator) we are considering only 20 random data points instead of all points whereas in greedy we have considered all the points (numerator), hence the approximation factor may not be that accurate and is not a good indicator (it is actually wrong by definition as we are applying the two algorithms on different data sets). This factor may come out to be greater than 2 sometimes.


Note - All the above values may vary on each execution of algorithm since it involves some randomness, however the observations remain the same.

### Challenges Faced 

- For some data points, some values are NULL / missing.
- To overcome this issue, I first took dimension-wise mean of the not NULL entries and then assigned this value to all the data points that had NULL in that dimension.

### Code -

The code can be found below (it uses the file que4_kaggle_dataset.csv downloaded from https://www.kaggle.com/arjunbhasin2013/ccdata)

In [11]:
import pandas as pd
import random
import math

# Taking values from the CSV file
data = pd.read_csv('que4_kaggle_dataset.csv', sep=',').values

# Dimension and Length of data
dimensions = len(data[0])-1
data_len = len(data)

In [12]:
# Function to set NULL values in data to mean of that column
def handle_null_values():    
    for j in range(1, dimensions+1):
        
        # finding mean
        sum_of_all_values = 0
        count = 0
        for i in range(data_len):
            if not math.isnan(data[i][j]):
                    sum_of_all_values+=data[i][j]
                    count+=1
                    
        mean = sum_of_all_values/count
                    
        for i in range(data_len):
            if math.isnan(data[i][j]):
                    data[i][j] = mean

# Function to calculate L2 norm distance between two vectors x and y                
def L2_norm_distance(x, y):        
    distance = 0.0    
    for i in range(1, len(x)):
        distance+=pow(x[i]-y[i], 2)                    
    return pow(distance,0.5)

# Function to normalize jth dimension
def normalize_dimension(j):
    # jth dimension 
    MIN = float('inf')
    MAX = float('-inf')
    for i in range(data_len):
        MIN = min(data[i][j], MIN)
        MAX = max(data[i][j], MAX)    
    for i in range(data_len):        
        if MAX-MIN > 0:
            data[i][j] =  (data[i][j]-MIN)/(MAX-MIN)

In [13]:
#Function that returns indices of k-centers by using greedy k-center algorithm for given indices of data points
def find_k_centers(k, data_point_indices):
    
    # To store indices ( data points) of current centers
    current_centers = [data_point_indices[random.randrange(0, len(data_point_indices)-1)]] # Choosing a random start center
    
    # Loop which number of centers is less than k
    while len(current_centers) < k:
                
        new_center = 0 # To store index of new center
        new_center_distance = float('-inf') # To store distance to new center
                
        for i in data_point_indices:
                    
            distance_to_closest_center = float('inf') # to store distance of ith data point to its closest center
            
            for j in range(len(current_centers)):                                            
                distance_to_closest_center = min(distance_to_closest_center, 
                                                 L2_norm_distance(data[current_centers[j]], data[i]))                
            
            # if the ith point has distance more than distance for new center then assign ith point as new center and update its distance
            if new_center_distance < distance_to_closest_center:
                new_center_distance = distance_to_closest_center
                new_center = i                
                
        current_centers.append(new_center)            
    
    return current_centers

In [14]:
# Function to calculate k-center cost with given number of centers, given k centers and given list of data points
def calculate_k_center_cost(number_of_centers, k_centers, data_point_indices):        
    
    k_center_cost = float('-inf')
    
    for i in data_point_indices:
        
        distance_to_closest_center = float('inf')
        
        # Consider only first number_of_centers centers in the given centers
        for j in range(number_of_centers):
            
            distance_to_closest_center = min(distance_to_closest_center, L2_norm_distance(data[k_centers[j]], data[i]))
            
        k_center_cost = max(k_center_cost, distance_to_closest_center)
        
    return k_center_cost

In [15]:
# Replace NULL values by mean of that column
handle_null_values()

# Normalize all dimensions
for j in range(1, dimensions+1):
    normalize_dimension(j)          
    
all_data_point_indices = [i for i in range(data_len)]

In [16]:
# find 10 centers by greedy algorithm
k_centers_by_algo = find_k_centers(10, all_data_point_indices) # Finding 10 centers before hand so we do not need to find again for 2 or 4, we can pick the first 2 or first 4 for these.
k_center_cost_by_greedy_algo_k_2 = calculate_k_center_cost(2, k_centers_by_algo, all_data_point_indices)
k_center_cost_by_greedy_algo_k_4 = calculate_k_center_cost(4, k_centers_by_algo, all_data_point_indices)
k_center_cost_by_greedy_algo_k_10 = calculate_k_center_cost(10, k_centers_by_algo, all_data_point_indices)

In [17]:
# Calculation for optimal cost

# Suppose, n = no. of data points, d = dimension of each data point
# For optimal algorithm with k = 2, the time complexity would be O((n^3)*k*d) 
# For optimal algorithm with k = 4, the time complexity would be O((n^5)*k*d) 
# Hence, applying only on very few random points instead of n

number_of_points_to_apply_optimal_algo = 20 # This ideally should be equal to n but program will take many hours/days to finish, hence very small value is taken.

# Picking up number_of_points_to_apply_optimal_algo many random points to apply optimal algorithm
data_point_indices = random.sample(all_data_point_indices, number_of_points_to_apply_optimal_algo)

# Optimal for k = 2
optimal_cost_for_k_2 = float('inf')

# Try all possible points as center from data_point_indices
for i in range(len(data_point_indices)):
    for j in range(i+1, len(data_point_indices)):           
        first_center_index = data_point_indices[i]
        second_center_index = data_point_indices[j]        
        cost_with_these_centers = calculate_k_center_cost(2, [first_center_index, second_center_index], data_point_indices)                                        
        optimal_cost_for_k_2 = min(cost_with_these_centers, optimal_cost_for_k_2)                

# Optimal for k = 4
optimal_cost_for_k_4 = float('inf')

# Try all possible points as center from data_point_indices
for i in range(len(data_point_indices)):
    for j in range(i+1, len(data_point_indices)):   
        for k in range(j+1, len(data_point_indices)):   
            for l in range(k+1, len(data_point_indices)):           
                first_center_index = data_point_indices[i]
                second_center_index = data_point_indices[j]
                third_center_index = data_point_indices[k]
                fourth_center_index = data_point_indices[l]
                cost_with_these_centers = calculate_k_center_cost(4, [first_center_index, second_center_index, third_center_index, fourth_center_index], data_point_indices)                                        
                optimal_cost_for_k_4 = min(cost_with_these_centers, optimal_cost_for_k_4)           
                
                
# Also, applying greedy on these small number of points
k_centers_by_algo_for_smaller_n = find_k_centers(4, data_point_indices)
k_center_cost_by_greedy_algo_k_2_for_smaller_n = calculate_k_center_cost(2, k_centers_by_algo_for_smaller_n, data_point_indices)
k_center_cost_by_greedy_algo_k_4_for_smaller_n = calculate_k_center_cost(4, k_centers_by_algo_for_smaller_n, data_point_indices)

In [18]:
approximation_factor_k_2_for_smaller_n = (k_center_cost_by_greedy_algo_k_2_for_smaller_n/optimal_cost_for_k_2)
approximation_factor_k_4_for_smaller_n = (k_center_cost_by_greedy_algo_k_4_for_smaller_n/optimal_cost_for_k_4)
approximation_factor_k_2 = (k_center_cost_by_greedy_algo_k_2/optimal_cost_for_k_2)
approximation_factor_k_4 = (k_center_cost_by_greedy_algo_k_4/optimal_cost_for_k_4)

In [24]:
print("Customer ids of choosen 10-centers by greedy k-center algorithm in all data points are : \n")

for center in k_centers_by_algo:
    print(data[center][0], end=" ")    
    
print("\n")

print("Customer ids of choosen 4-centers by greedy k-center algorithm in "+str(number_of_points_to_apply_optimal_algo)+" random data points are : \n")

for center in k_centers_by_algo_for_smaller_n:
    print(data[center][0], end=" ")    
    
print("\n")

       
print("K-center cost by greedy algorithm for k=2 with all points is "+str(k_center_cost_by_greedy_algo_k_2)+"\n")
print("K-center cost by greedy algorithm for k=4 with all points is "+str(k_center_cost_by_greedy_algo_k_4)+"\n")
print("K-center cost by greedy algorithm for k=10 with all points is "+str(k_center_cost_by_greedy_algo_k_10)+"\n")
print("K-center cost by greedy algorithm for k=2 with "+str(number_of_points_to_apply_optimal_algo)+" points is "+str(k_center_cost_by_greedy_algo_k_2_for_smaller_n)+"\n")
print("K-center cost by greedy algorithm for k=4 with "+str(number_of_points_to_apply_optimal_algo)+" points is "+str(k_center_cost_by_greedy_algo_k_4_for_smaller_n)+"\n")

    
    
print("K-center cost by optimal algorithm for k=2 with "+str(number_of_points_to_apply_optimal_algo)+" points is "+str(optimal_cost_for_k_2)+"\n")
print("K-center cost by optimal algorithm for k=4 with "+str(number_of_points_to_apply_optimal_algo)+" points is "+str(optimal_cost_for_k_4)+"\n")


print("Approximation factor which has been calculated by applying greedy algorithm on "+str(number_of_points_to_apply_optimal_algo)+" random data points and optimal algorithm on those 20 data points\n")

print("for k = 2 : "+str(approximation_factor_k_2_for_smaller_n)+"\n")
print("for k = 4 : "+str(approximation_factor_k_4_for_smaller_n)+"\n")

print("As expected this value lies in range [1, 2]\n")


print("Approximation factor which has been calculated by applying greedy algorithm on all data points and optimal algorithm on random 20 data points\n")

print("for k = 2 : "+str(approximation_factor_k_2)+"\n")
print("for k = 4 : "+str(approximation_factor_k_4)+"\n")


print("In this case, since for calculation of optimal cost we are considering only "+str(number_of_points_to_apply_optimal_algo)+" random data points instead of all points whereas in greedy we have considered all the points, hence the approximation factor may not be that accurate and is not a good indicator.")

Customer ids of choosen 10-centers by greedy k-center algorithm in all data points are : 

C11992 C11300 C13663 C14338 C12226 C13762 C10523 C12056 C17325 C13127 

Customer ids of choosen 4-centers by greedy k-center algorithm in 20 random data points are : 

C17744 C12226 C13045 C10797 

K-center cost by greedy algorithm for k=2 with all points is 2.0106026585562478

K-center cost by greedy algorithm for k=4 with all points is 1.789436782259935

K-center cost by greedy algorithm for k=10 with all points is 1.4984577152624328

K-center cost by greedy algorithm for k=2 with 20 points is 1.679758268511973

K-center cost by greedy algorithm for k=4 with 20 points is 1.2612759905672448

K-center cost by optimal algorithm for k=2 with 20 points is 1.258411509001605

K-center cost by optimal algorithm for k=4 with 20 points is 1.00772537014411

Approximation factor which has been calculated by applying greedy algorithm on 20 random data points and optimal algorithm on those 20 data points

fo